In [1]:
import subprocess, re, copy
import pandas as pd

# Tools

In [2]:
def saveCSVNames(folderName):
    with open('csvname_' + folderName + '.txt', 'w') as f:
        command = "ssh gpu2 'ls container/3/OpenFace/build/" + folderName + " | grep csv'"
        f.write(subprocess.getoutput(command))
        
def makeFolderForCSV(name):
    command = "mkdir " + name
    subprocess.getoutput(command)

def getCSVNames(fileName):
    temp = []
    with open(fileName, 'r') as f:
        temp += f.readlines()

    result = []
    for i in temp:
        result += [i.split('\n').pop(0)]
    return result
        
def getFolderNames():
    f = open('folders.txt', 'r')
    folderNames = []
    for i in f.readlines():
        temp = i.replace('\n', '')
        folderNames += [temp]
    return folderNames
# test = getCSVNames('csvname_PrototypeFacialExpressions.txt')
# print(test)

# get all folder names

In [3]:
# get all types of folders
def get_all_folders():
    command = "ssh gpu2 'ls container/3/Nikola/'"
    subprocess.getoutput(command)
    with open('folders.txt', 'w') as f:
        print(subprocess.getoutput(command))
        f.write(subprocess.getoutput(command))
        f.close()

# scp all csv


In [5]:
folderNames = getFolderNames()

for i in folderNames:
    saveCSVNames(i)
    makeFolderForCSV(i)
    csvnames = getCSVNames('csvname_' + i + '.txt')
    for j in csvnames:
        command = 'scp gpu2:container/3/OpenFace/build/' + i + '/' + j + ' ./' + i + '/'
        subprocess.getoutput(command)


# Plot and save

In [ ]:
folderNames = getFolderNames()

csv_data_netual = pd.read_csv("./PrototypeFacialExpressions/2021_05_21_16_40_08_netural.csv", index_col='frame')


for i in folderNames:
    csvnames = getCSVNames('csvname_' + i + '.txt')
    AUs = {}
    keys = []
    for j in csvnames:
        temp = str(re.split(r"[_.]", j)[6])
        keys += [temp]


    for j in range(len(keys)):
        AUs[keys[j]] = pd.read_csv(i + '/' + csvnames[j], index_col='frame')
    

    # Plot and save 
    tempAUs = copy.deepcopy(AUs)

    # subtract neural
    for k,v in tempAUs.items():
        tempAUs[k] -= csv_data_netual
    
    for k,v in tempAUs.items():
        temp_r = tempAUs[k].iloc[:, -35:-18]
        temp_c = tempAUs[k].iloc[:, -18:]

        plt_r = temp_r.plot(figsize=(18, 13))
        fig_r = plt_r.get_figure()
        fig_r.patch.set_facecolor('white')
        fig_r_Name = i + '/' + k + '_r.png'
        fig_r.savefig(fig_r_Name, dpi=300)
        fig_r.clf() # not show

        axs = temp_c.plot(figsize=(5, 10), subplots=True)
        fig_c = axs[0].get_figure()
        fig_c.patch.set_facecolor('white')
        fig_c_Name = i + '/' + k + '_c.png'
        fig_c.savefig(fig_c_Name, dpi=300)
        fig_c.clf() # not show
    


In [ ]:
for i in range(len(keys)):
    AUs[keys[i]] = pd.read_csv(csvnames[i], index_col='frame')

# Normalization

In [ ]:
csv_data_netual = pd.read_csv("./PrototypeFacialExpressions/2021_05_21_16_40_08_netural.csv", index_col='frame')
import pandas as pd
from sklearn import preprocessing

x = csv_data_netual.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
print(df)

# AU matrix

In [5]:
folderNames = getFolderNames()
print(folderNames)

['PrototypeAU', 'PrototypeFacialExpressions', 'hotExpressions']


In [6]:
i = folderNames[0]
csvnames = getCSVNames('csvname_' + i + '.txt')
AUs = {}
keys = []
for j in csvnames:
    temp = str(re.split(r"[_.]", j)[6])
    keys += [temp]


for j in range(len(keys)):
    AUs[keys[j]] = pd.read_csv(i + '/' + csvnames[j], index_col='frame')

print('keys', keys)
print("AUs", AUs)

IndentationError: unexpected indent (<ipython-input-6-d062e29fe3bb>, line 3)